# 32 BIT CIFTLI

In [ ]:
import numpy as np
import time

# Generate probabilities
numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
numbers_for_ones = np.round(numbers_for_ones, 4)

numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
numbers_for_zeros = np.round(numbers_for_zeros, 4)

random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

combined_numbers = np.concatenate((numbers_for_ones, numbers_for_zeros))
combined_numbers = np.insert(combined_numbers, 0, random_elements[0])
combined_numbers = np.append(combined_numbers, random_elements[1])

np.random.shuffle(combined_numbers[1:-1])

# Calculate the bit string
bit_string = ''.join(['1' if num > 0.5 else '0' for num in combined_numbers])

# Calculate the probabilities for each 2-bit string for all pairs
bit_string_probs_list = []
for i in range(0, len(combined_numbers) - 1, 2):
    p1 = combined_numbers[i]
    p2 = combined_numbers[i + 1]
    
    prob_00 = round((1 - p1) * (1 - p2), 4)
    prob_01 = round((1 - p1) * p2, 4)
    prob_10 = round(p1 * (1 - p2), 4)
    prob_11 = round(p1 * p2, 4)
    
    bit_string_probs = {
        'tuple_index': i // 2,
        '00': prob_00,
        '01': prob_01,
        '10': prob_10,
        '11': prob_11
    }
    
    bit_string_probs_list.append(bit_string_probs)


print('Probabilities:', combined_numbers)
print('Bit string:', bit_string)
print('Probabilities for each 2-bit string:')
for bit_string_probs in bit_string_probs_list:
    print(bit_string_probs)
print('number for ones =', random_elements)

# Function to find the target string with tries and calculate the finding time
def find_target_string(target_string, bit_string_probs_list, max_errors=1):
    start_time = time.time()
    attempts = 0
    
    def backtrack(current_string="", index=0, errors=0):
        nonlocal attempts
        attempts += 1
        
        if index == len(bit_string_probs_list):
            if errors <= max_errors:
                return current_string
            else:
                return None
        
        current_probs = bit_string_probs_list[index]
        sorted_tuples = sorted(current_probs.items(), key=lambda item: item[1], reverse=True)
        
        for bit_tuple, _ in sorted_tuples:
            if bit_tuple == 'tuple_index':
                continue
            
            next_string = current_string + bit_tuple
            expected_bits = target_string[index*2:(index*2)+2]
            
            new_errors = errors + sum(1 for a, b in zip(bit_tuple, expected_bits) if a != b)
            
            if new_errors <= max_errors:
                result = backtrack(next_string, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
        
        return None
    
    result_string = backtrack()
    end_time = time.time()
    elapsed_time = end_time - start_time
    return result_string, elapsed_time, attempts

# Example target string (for demonstration purposes)
# Convert the string to a list
target_list = list(bit_string)

changed_index_list = [2, 10, 20, 30, 15]  # Example of multiple bit changes
for index in changed_index_list:
    target_list[index] = '1' if target_list[index] == '0' else '0'

# Convert the list back to a string
target_string = ''.join(target_list)

# Find the target string and calculate the finding time
found_string, finding_time, total_attempts = find_target_string(target_string, bit_string_probs_list, max_errors=11)

# Output the results
print("\nTarget string:", target_string)
print("Found string:", found_string)
if found_string == target_string:
    print("Target string found successfully!")
print("Time taken to find the target string:", finding_time, "seconds")
print("Total attempts:", total_attempts)


# 32 BIT TEKLI

In [19]:
import numpy as np
import time
from itertools import combinations

# Generate probabilities
numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
numbers_for_ones = np.round(numbers_for_ones, 4)

numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
numbers_for_zeros = np.round(numbers_for_zeros, 4)

random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

np.random.shuffle(probabilities_for_each_bit[1:-1])

# Calculate the bit string
bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

target_list = list(bit_string)


#changed_index_list = np.random.choice(range(32), 5, replace=False)
changed_index_list = [30]  
for index in changed_index_list:
    target_list[index] = '1' if target_list[index] == '0' else '0'

# Convert the list back to a string
target_string = ''.join(target_list)

# Reverse the target string
reversed_target_string = target_string[::-1]



# Function to find the target string with tries and calculate the finding time
def find_target_string(target_string, combined_numbers, max_errors=1):
    start_time = time.time()
    attempts = 0
    
    def backtrack(current_string="", index=0, errors=0):
        nonlocal attempts
        attempts += 1
        
        if index == len(combined_numbers):
            if errors <= max_errors:
                return current_string
            else:
                return None
        
        prob = combined_numbers[index]
        bit = '1' if prob > 0.5 else '0'
        expected_bit = target_string[index]
        
        new_errors = errors + (1 if bit != expected_bit else 0)
        
        # Try the current bit
        if new_errors <= max_errors:
            result = backtrack(current_string + bit, index + 1, new_errors)
            if result is not None and result == target_string:
                return result
        
        # Try the flipped bit
        bit = '0' if bit == '1' else '1'
        new_errors = errors + (1 if bit != expected_bit else 0)
        
        if new_errors <= max_errors:
            result = backtrack(current_string + bit, index + 1, new_errors)
            if result is not None and result == target_string:
                return result
        
        return None
    
    result_string = backtrack()
    end_time = time.time()
    elapsed_time = end_time - start_time
    return result_string, elapsed_time, attempts

# Find the reversed target string and calculate the finding time
found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=11)

#Output the results
print("Combined numbers:", probabilities_for_each_bit)
print("Bit string:", bit_string)

print("\nTarget string:", target_string)
print("Found string:", found_string)
if found_string == target_string:
    print("Target string found successfully!")
else:
    print("Failed to find the target string.")
print("Time taken to find the target string:", finding_time, "seconds")
print("Total attempts:", total_attempts)


Combined numbers: [0.7067 0.72   0.2827 0.688  0.7093 0.2907 0.3173 0.712  0.28   0.32
 0.2853 0.312  0.6827 0.3147 0.3013 0.6933 0.7147 0.304  0.7173 0.2987
 0.3093 0.296  0.6853 0.7013 0.288  0.6907 0.6987 0.2933 0.68   0.704
 0.3067 0.696 ]
Bit string: 11011001000010011010001101101101

Target string: 11011001000010011010001101101111
Found string: 11011001000010011010001101101111
Target string found successfully!
Time taken to find the target string: 2.2649765014648438e-05 seconds
Total attempts: 36


In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 5, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=9)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_5error.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(100):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 4, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=9)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_4error.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(100):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 3, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=9)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_3error.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(100):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 2, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=9)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_2error.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(100):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 16)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 16)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 1, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=9)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_1error.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(100):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

# 64 BIT CIFTLI

In [ ]:
import numpy as np
import time

# Generate probabilities for bit values
numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
numbers_for_ones = np.round(numbers_for_ones, 4)

numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
numbers_for_zeros = np.round(numbers_for_zeros, 4)

random_elements = np.random.choice(numbers_for_ones, 4, replace=False)
numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

combined_numbers = np.concatenate((numbers_for_ones, numbers_for_zeros))
combined_numbers = np.insert(combined_numbers, 0, random_elements[0])
combined_numbers = np.append(combined_numbers, random_elements[1])
combined_numbers = np.append(combined_numbers, random_elements[2:])
np.random.shuffle(combined_numbers[1:-1])

# Generate the initial bit string based on probabilities
bit_string = ''.join(['1' if num > 0.5 else '0' for num in combined_numbers])

# Calculate probabilities for each 2-bit string combination
bit_string_probs_list = []
for i in range(0, len(combined_numbers) - 1, 2):
    p1 = combined_numbers[i]
    p2 = combined_numbers[i + 1]
    
    prob_00 = round((1 - p1) * (1 - p2), 4)
    prob_01 = round((1 - p1) * p2, 4)
    prob_10 = round(p1 * (1 - p2), 4)
    prob_11 = round(p1 * p2, 4)
    
    bit_string_probs = {
        'tuple_index': i // 2,
        '00': prob_00,
        '01': prob_01,
        '10': prob_10,
        '11': prob_11
    }
    
    bit_string_probs_list.append(bit_string_probs)

# Recursive function to find the target string
def find_target_string(target_string, bit_string_probs_list, max_errors=1):
    start_time = time.time()
    attempts = 0
    
    def backtrack(current_string="", index=0, errors=0):
        nonlocal attempts
        attempts += 1
        
        if index == len(bit_string_probs_list):
            if errors <= max_errors:
                return current_string if current_string == target_string else None
            else:
                return None
        
        current_probs = bit_string_probs_list[index]
        sorted_tuples = sorted(current_probs.items(), key=lambda item: item[1], reverse=True)
        
        for bit_tuple, _ in sorted_tuples:
            if bit_tuple == 'tuple_index':
                continue
            
            next_string = current_string + bit_tuple
            expected_bits = target_string[index*2:(index*2)+2]
            
            new_errors = errors + sum(1 for a, b in zip(bit_tuple, expected_bits) if a != b)
            
            if new_errors <= max_errors:
                result = backtrack(next_string, index + 1, new_errors)
                if result is not None:
                    return result
        
        return None
    
    result_string = backtrack()
    end_time = time.time()
    elapsed_time = end_time - start_time
    return result_string, elapsed_time, attempts

# Example target string with some errors introduced
target_list = list(bit_string)
changed_index_list = [2, 5, 8, 17]  # Example of multiple bit changes
for index in changed_index_list:
    target_list[index] = '1' if target_list[index] == '0' else '0'

# Convert the list back to a string
target_string = ''.join(target_list)

# Find the target string and calculate the finding time
found_string, finding_time, total_attempts = find_target_string(target_string, bit_string_probs_list, max_errors=10)

# Output the results
print("Combined numbers:", combined_numbers)
print("Bit string:", bit_string)
print("Probabilities for each 2-bit string tuple:")
for probs in bit_string_probs_list:
    tuple_index = probs.pop('tuple_index')
    print(f"Tuple {tuple_index}: {probs}")

print("\nTarget string:", target_string)
print("Found string:", found_string)
print("Time taken to find the target string:", finding_time, "seconds")
print("Total attempts:", total_attempts)


# 128 BIT TEKLI

In [ ]:
import numpy as np
import time

# Generate probabilities
numbers_for_ones = np.linspace(0.6800, 0.7200, 64)
numbers_for_ones = np.round(numbers_for_ones, 4)

numbers_for_zeros = np.linspace(0.2800, 0.3200, 64)
numbers_for_zeros = np.round(numbers_for_zeros, 4)

random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

np.random.shuffle(probabilities_for_each_bit[1:-1])

# Calculate the bit string
bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

# Function to find the target string with tries and calculate the finding time
def find_target_string(target_string, combined_numbers, max_errors=1):
    start_time = time.time()
    attempts = 0
    
    def backtrack(current_string="", index=0, errors=0):
        nonlocal attempts
        attempts += 1
        
        if index == len(combined_numbers):
            if errors <= max_errors:
                return current_string
            else:
                return None
        
        prob = combined_numbers[index]
        bit = '1' if prob > 0.5 else '0'
        expected_bit = target_string[index]
        
        new_errors = errors + (1 if bit != expected_bit else 0)
        
        # Try the current bit
        if new_errors <= max_errors:
            result = backtrack(current_string + bit, index + 1, new_errors)
            if result is not None and result == target_string:
                return result
        
        # Try the flipped bit
        bit = '0' if bit == '1' else '1'
        new_errors = errors + (1 if bit != expected_bit else 0)
        
        if new_errors <= max_errors:
            result = backtrack(current_string + bit, index + 1, new_errors)
            if result is not None and result == target_string:
                return result
        
        return None
    
    result_string = backtrack()
    end_time = time.time()
    elapsed_time = end_time - start_time
    return result_string, elapsed_time, attempts

# Example target string (for demonstration purposes)
# Convert the string to a list
target_list = list(bit_string)

changed_index_list = [2, 5, 35, 45]  # Example of multiple bit changes
for index in changed_index_list:
    target_list[index] = '1' if target_list[index] == '0' else '0'

# Convert the list back to a string
target_string = ''.join(target_list)

# Find the target string and calculate the finding time
found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=6.4)

# Output the results
print("Combined numbers:", probabilities_for_each_bit)
print("Bit string:", bit_string)

print("\nTarget string:", target_string)
print("Found string:", found_string)
if found_string == target_string:
    print("Target string found successfully!")
else:
    print("Failed to find the target string.")
print("Time taken to find the target string:", finding_time, "seconds")
print("Total attempts:", total_attempts)

# 64BIT TESTLER

## max 22

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 10, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=22)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_65.625_accuracy_10error_64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 8, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=22)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_65.625_accuracy_8error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 6, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=22)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_65.625_accuracy_6error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 4, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=22)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_65.625_accuracy_4error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 2, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=22)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_65.625_accuracy_2error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

## max 18

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 10, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=18)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_10error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 8, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=18)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_8error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 6, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=18)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_6error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 4, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=18)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_4error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 2, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=18)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_71.875_accuracy_2error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

## max 14

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 10, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=14)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_78.125_accuracy_10error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 8, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=14)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_78.125_accuracy_8error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 6, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=14)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_78.125_accuracy_6error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 4, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=14)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_78.125_accuracy_4error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")

In [ ]:
import numpy as np
import time


def run_experiment():
    # Generate probabilities
    numbers_for_ones = np.linspace(0.6800, 0.7200, 32)
    numbers_for_ones = np.round(numbers_for_ones, 4)

    numbers_for_zeros = np.linspace(0.2800, 0.3200, 32)
    numbers_for_zeros = np.round(numbers_for_zeros, 4)

    random_elements = np.random.choice(numbers_for_ones, 2, replace=False)
    numbers_for_ones = np.setdiff1d(numbers_for_ones, random_elements)

    probabilities_for_each_bit = np.concatenate((numbers_for_ones, numbers_for_zeros))
    probabilities_for_each_bit = np.insert(probabilities_for_each_bit, 0, random_elements[0])
    probabilities_for_each_bit = np.append(probabilities_for_each_bit, random_elements[1])

    np.random.shuffle(probabilities_for_each_bit[1:-1])

    # Calculate the bit string
    bit_string = ''.join(['1' if num > 0.5 else '0' for num in probabilities_for_each_bit])

    target_list = list(bit_string)
    changed_index_list = np.random.choice(range(32), 2, replace=False)
    for index in changed_index_list:
        target_list[index] = '1' if target_list[index] == '0' else '0'
    target_string = ''.join(target_list)


    # Function to find the target string with tries and calculate the finding time
    def find_target_string(target_string, combined_numbers, max_errors=1):
        start_time = time.time()
        attempts = 0
        
        def backtrack(current_string="", index=0, errors=0):
            nonlocal attempts
            attempts += 1
            
            if index == len(combined_numbers):
                if errors <= max_errors:
                    return current_string
                else:
                    return None
            
            prob = combined_numbers[index]
            bit = '1' if prob > 0.5 else '0'
            expected_bit = target_string[index]
            
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            # Try the current bit
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            # Try the flipped bit
            bit = '0' if bit == '1' else '1'
            new_errors = errors + (1 if bit != expected_bit else 0)
            
            if new_errors <= max_errors:
                result = backtrack(current_string + bit, index + 1, new_errors)
                if result is not None and result == target_string:
                    return result
            
            return None
        
        result_string = backtrack()
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result_string, elapsed_time, attempts

    # Find the reversed target string and calculate the finding time
    found_string, finding_time, total_attempts = find_target_string(target_string, probabilities_for_each_bit, max_errors=14)
    return finding_time, total_attempts


total_time = 0
total_attempts = 0

# Open the text file
with open("results_78.125_accuracy_2error64bit.txt", "w") as file:
    # Run the experiment 100 times
    for i in range(10):
        # Run the experiment
        finding_time, attempts = run_experiment()

        # Update the total time and attempts
        total_time += finding_time
        total_attempts += attempts

        # Write the results for this run to the text file
        file.write(f"Run {i+1}: {finding_time} seconds, {attempts} attempts\n")

    # Calculate the average time and attempts
    average_time = total_time / 100
    average_attempts = total_attempts / 100

    # Write the averages to the text file
    file.write(f"\nAverage time: {average_time} seconds\n")
    file.write(f"Average attempts: {average_attempts}\n")